In [1]:
import sys
import time
import ee
import pprint
from openet.ssebop.image import Image

In [2]:
try:
    ee.Initialize()
    print('worked')
except ee.EEException:
    ee.Authenticate()
    ee.Initialize()
    print('worked second try')

worked


In [3]:
ndvi_palette = ['#EFE7E1', '#003300']
et_palette = [
    'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
    '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']
image_size = 768

In [4]:
def ee_task_start(task, n=10):
    """Make an exponential backoff Earth Engine request"""
    output = None
    for i in range(1, n):
        try:
            task.start()
            break
        except Exception as e:
            time.sleep(i ** 2)
    return task

In [5]:

# landsat_id = 'LC08_021033_20150115' # Indianapolis, IN, Winter
# landsat_id = 'LE07_021033_20150718' # Indianapolis, IN, Summer
# landsat_id = 'LE07_028041_20150124' # RGValley, TX, Winter
# landsat_id = 'LE07_028041_20150703' # RGValley, TX, Summer
# landsat_id = 'LE07_032038_20150104' # West TX Winter
# landsat_id = 'LC08_032038_20150707' # West TX Summer
# landsat_id = 'LC08_019036_20150117' # Alpharreta GA, Winter
# landsat_id = 'LC08_019036_20150712' # Alpharreta GA, Summer

# landsat_id = 'LC08_042030_20150704' # Western Idaho Summer
# landsat_id = 'LC08_042030_20150102' # Western Idaho Winter
# landsat_id = 'LE07_043035_20150101' # Salinas Ca, Winter
# landsat_id = 'LE07_043035_20150712' # Salinas Ca, Summer
# landsat_id = 'LC08_043035_20150720' # Salinas Ca, Summer
# landsat_id = 'LC08_044035_20150116' # Salinas Ca, Winter

# image_names = ['LC08_021033_20150115','LC08_032038_20150707','LC08_019036_20150117',
#                'LC08_019036_20150712','LC08_043035_20150720', 'LC08_044035_20150116']
image_names = ['LC08_043035_20180610']
image_paths = []
for img in image_names:
    img_lst = img.split('_')
    root = 'LANDSAT/{}/C02/T1_L2'.format(img_lst[0])
    img_name = f'{img_lst[0]}_{img_lst[1]}_{img_lst[2]}'
    img_path = f'{root}/{img_name}'
    image_paths.append(img_path)


In [6]:
print(image_paths)

['LANDSAT/LC08/C02/T1_L2/LC08_043035_20180610']


In [7]:
for n, p in zip(image_names, image_paths):
    # landsat images
    print(f'doing {n}')
    landsat_img = ee.Image(p)
    landsat_crs = landsat_img.select('SR_B3').projection().getInfo()['crs']
    landsat_region = landsat_img.geometry().bounds(1, 'EPSG:4326')
    landsat_dt = ee.Date(landsat_img.get('system:time_start'))
    # Build the SSEBop object from the Landsat image
#     test = Image(None)
#     print(test.salutations)
    model_obj = Image.from_landsat_c2_sr(
        landsat_img,
        et_reference_source='IDAHO_EPSCOR/GRIDMET', 
        et_reference_band='etr', 
        et_reference_factor=0.85,
        et_reference_resample='nearest',
        tcorr_source='GRIDDED_COLD',
    )
    # Initiate the Lazy Attributes
    tcorr = model_obj.tcorr_gridded_cold
    ETf = model_obj.et_fraction
    
    name_root = f'Tcorr_26032020_rn64_{n}'
    tcorr_name =  f'{name_root}'
    etf_name = f'etf_{name_root}'
    eta_name = f'eta_{name_root}'
    
    task = ee.batch.Export.image.toAsset(image=tcorr , description=tcorr_name, assetId=f'users/gbrlparrish/gridc_march/{tcorr_name}', region=landsat_region)
    print('starting tcorr export')
    
    # TODO - does the Tcorr 
    ee_task_start(task=task)
    print('task submitted')
    task = ee.batch.Export.image.toAsset(image=ETf, description=etf_name, assetId=f'users/gbrlparrish/gridc_march/{etf_name}', region=landsat_region)
    
    # start the task
    print('starting ETf export')
    ee_task_start(task=task)
    print('task submitted')

doing LC08_043035_20180610
starting tcorr export
task submitted
starting ETf export
task submitted


In [8]:

# # matching ETo image
# eto_img = ee.Image('projects/usgs-ssebop/pet/gridmet_median_v1/doy196')


In [9]:
# ndvi_image = model_obj.ndvi
# print(ndvi_image.getInfo())

In [10]:
# # === USER ====
# attempt = 'I'
# level = 'GS_Alg'
# name_root = f'rn{level}_vers{attempt}'

# tcorr_name =  f'a_test_tcorr_{name_root}'
# etf_name = f'a_test_etf_{name_root}'
# eta_name = f'a_test_eta_{name_root}'

In [11]:
# # try to export and see if the image looks like expected.
# # task = ee.batch.Export.image.toDrive(image=tcorr_image, description='CFact_TestGTE', region=landsat_region)
# task = ee.batch.Export.image.toAsset(image=tcorr , description=tcorr_name, assetId=f'users/gbrlparrish/gridCForMattS/{tcorr_name}', region=landsat_region)
# # start the tasktest
# print('starting')
# ee_task_start(task=task)
# print('task submitted')
# # task2 = ee.batch.Export.image.toAsset(image= tcorr_rn02_hot_testI, description='tcorr_rn02_hot_testI', assetId='users/gbrlparrish/griddedC/tcorr_rn02_hot_testI', region=landsat_region)
# # # start the task
# # print('starting task 2')
# # ee_task_start(task=task2)
# # print('task 2 submitted')
# # # # 

In [12]:
# ETf = model_obj.et_fraction
# # make ETa by multiplying by gridmet ETo v1
# ETa = ETf.multiply(eto_img)

In [13]:
# # try to export EToF (Doesn't work. Error is "Error: Too many input pixels per output pixel. Need 27778, but only 64 allowed." Like Whaaaaat)
# task = ee.batch.Export.image.toAsset(image=ETf, description=etf_name, assetId=f'users/gbrlparrish/griddedC/{etf_name}', region=landsat_region)
# # start the task
# print('starting')
# ee_task_start(task=task)
# print('task submitted')
# # ============================
# task = ee.batch.Export.image.toAsset(image=ETa, description=eta_name, assetId=f'users/gbrlparrish/griddedC/{eta_name}', region=landsat_region)
# # start the task
# print('starting')
# ee_task_start(task=task)
# print('task submitted')

# # # TODO - export as asset and then display in the notebook.
# # # # # =========================================================================
# # # # # Try to show EToF and it also doesn't work
# # # image_url = model_obj.et_fraction\
# # #     .getThumbURL({'min': 0, 'max': 1.2, 'palette': et_palette, 
# # #                   'region': landsat_region, 'dimensions': image_size})
# # # Image(image_url, embed=True, format='png')